# 🔄 Spotify Library Sync

This notebook downloads your Spotify library and saves it locally for offline analysis.

**What it does:**
- ✅ Fetches all your playlists (owned only)
- ✅ Fetches your Liked Songs (❤️ master playlist)
- ✅ Downloads track and artist metadata
- ✅ Saves everything to `../data/` as parquet files
- ✅ Incremental updates (only fetches changes)

**Run this first!** Then use `02_analyze_library.ipynb` for analysis.

**💡 Tip:** For automated daily syncs, use `scripts/sync.py` instead (configured via cron job). See `README.md` for details.

## 1️⃣ Setup

Install dependencies and configure credentials.

In [12]:
# Install dependencies (run once)
%pip install -q pandas spotipy pyarrow tqdm python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [13]:
# Add project to path
import sys
from pathlib import Path

PROJECT_ROOT = Path("..").resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print(f"✅ Project root: {PROJECT_ROOT}")

✅ Project root: /Users/aryamaan/Desktop/Projects/spotim8


In [14]:
import os
from dotenv import load_dotenv

# Load credentials from ../.env file
env_path = PROJECT_ROOT / ".env"
if env_path.exists():
    load_dotenv(env_path)
    print(f"✅ Loaded credentials from {env_path}")
else:
    print(f"⚠️  No .env file found at {env_path}")
    print("   Create one with SPOTIPY_CLIENT_ID, SPOTIPY_CLIENT_SECRET, SPOTIPY_REDIRECT_URI")

# Verify credentials are set
client_id = os.environ.get("SPOTIPY_CLIENT_ID", "")
if client_id and client_id != "YOUR_CLIENT_ID":
    print(f"   Client ID: {client_id[:8]}...")
else:
    print("   ❌ SPOTIPY_CLIENT_ID not set!")

✅ Loaded credentials from /Users/aryamaan/Desktop/Projects/spotim8/.env
   Client ID: 8263fcc5...


## 2️⃣ Connect to Spotify

This will open a browser window for authentication on first run.

In [15]:
from spotim8 import Spotim8, set_response_cache
from spotim8.catalog import CacheConfig

# Data directory (stores downloaded data)
DATA_DIR = PROJECT_ROOT / "data"
DATA_DIR.mkdir(exist_ok=True)

# Enable API response caching to avoid rate limits
# Cached responses are reused for 1 hour
API_CACHE_DIR = DATA_DIR / ".api_cache"
set_response_cache(API_CACHE_DIR, ttl=3600)

# Initialize client with caching
sf = Spotim8.from_env(
    progress=True,
    cache=CacheConfig(dir=DATA_DIR)
)

print(f"✅ Connected to Spotify!")
print(f"📁 Data will be saved to: {DATA_DIR}")

📦 API response cache enabled: /Users/aryamaan/Desktop/Projects/spotim8/data/.api_cache (TTL: 3600s)
✅ Connected to Spotify!
📁 Data will be saved to: /Users/aryamaan/Desktop/Projects/spotim8/data


## 3️⃣ Sync Your Library

This fetches your playlists and tracks. First run may take a few minutes.

In [16]:
# Sync library (incremental - only fetches changes)
stats = sf.sync(
    owned_only=True,           # Only your playlists, not followed ones
    include_liked_songs=True   # Include Liked Songs as master playlist
)

print(f"\n📊 Sync complete!")

🔄 Starting library sync...


✅ All playlists up to date!
✅ Sync complete! Checked 221 playlists, updated 0, added 0 track entries

📊 Sync complete!


## 4️⃣ Build Full Data Tables

Now let's build all the detailed tables (tracks, artists, etc.)

In [17]:
# Fetch all data tables (uses cache if available)
print("📥 Building data tables...\n")

playlists = sf.playlists()
print(f"✅ Playlists: {len(playlists):,}")

playlist_tracks = sf.playlist_tracks()
print(f"✅ Playlist-track links: {len(playlist_tracks):,}")

tracks = sf.tracks()
print(f"✅ Unique tracks: {len(tracks):,}")

track_artists = sf.track_artists()
print(f"✅ Track-artist links: {len(track_artists):,}")

artists = sf.artists()
print(f"✅ Artists: {len(artists):,}")

# Build the wide table (everything joined)
library = sf.library_wide()
print(f"✅ Library wide table: {len(library):,} rows")

📥 Building data tables...

✅ Playlists: 650
✅ Playlist-track links: 49,370
✅ Unique tracks: 5,280
✅ Track-artist links: 8,517
✅ Artists: 2,618
✅ Library wide table: 49,856 rows


## 5️⃣ View Your Data

In [18]:
# Show status summary
sf.print_status()


        SPOTIM8 DATA STATUS
📁 Cache directory: /Users/aryamaan/Desktop/Projects/spotim8/data
👤 User: 31iol2qamank24owygxo7kpq533y
🕐 Last sync: 2025-12-25T22:09:40.777589+00:00

📊 Cached data:
   • Playlists: 650
   • Playlist tracks: 49,370
   • Unique tracks: 5,280
   • Track-artist links: 8,517
   • Artists: 2,618



In [19]:
# Preview playlists
print("📂 Your Playlists:")
playlists[["name", "track_count"]].head(15)

📂 Your Playlists:


,name,track_count
0,❤️ Liked Songs,5122
1,toNIKOfromAJ,30
2,AJFinds23,1233
3,AJFinds22,1150
4,AJFinds21,1310
5,AJamLatin,68
6,AJamIndie,268
7,AJamRock,132
8,AJamPop,356
9,AJamR&B/Soul,480


In [20]:
# Preview tracks
print("🎵 Sample Tracks:")
tracks[["name", "album_name", "popularity", "duration_ms"]].head(10)

🎵 Sample Tracks:


,name,album_name,popularity,duration_ms
0,Night Drive,Night Drive,65,220000
1,don't u know?,Missing in Action (The Return),57,176004
2,High No More,High No More,73,198845
3,Back On 74,Volcano,80,209482
4,Afraid To Feel,Afraid To Feel,77,177524
5,DARE (feat. Shaun Ryder & Roses Gabor),Demon Days,83,244999
6,Casio,For Ever,69,234369
7,Electric Feel,Oracular Spectacular,79,229640
8,Move Your Feet,D-D-Don't Don't Stop The Beat,71,181826
9,What I Might Do,What I Might Do (Radio Edit),53,195737


In [21]:
# Preview artists
print("🎤 Top Artists (by followers):")
artists.nlargest(10, "followers")[["name", "genres", "popularity", "followers"]]

🎤 Top Artists (by followers):


,name,genres,popularity,followers
1144,Arijit Singh,"[hindi pop, bollywood, desi, bangla pop]",92,168899201
226,Taylor Swift,[],100,148559087
759,Ed Sheeran,[soft pop],90,123907487
95,Billie Eilish,[],93,121060552
550,The Weeknd,[],96,115972161
178,Ariana Grande,[pop],95,108602111
656,Eminem,"[rap, hip hop]",91,106068116
1070,Bad Bunny,"[reggaeton, trap latino, urbano latino, latin]",98,105347204
44,Drake,[rap],98,105183779
756,Justin Bieber,[],94,85997799


## 6️⃣ Check Saved Files

In [22]:
# List saved files
print(f"📁 Files in {DATA_DIR}:\n")
for f in sorted(DATA_DIR.glob("*.parquet")):
    size_kb = f.stat().st_size / 1024
    print(f"   {f.name:30} {size_kb:>8.1f} KB")

📁 Files in /Users/aryamaan/Desktop/Projects/spotim8/data:

   artists.parquet                   198.6 KB
   library_wide.parquet             2089.7 KB
   playlist_tracks.parquet           641.8 KB
   playlists.parquet                 114.3 KB
   track_artists.parquet             217.5 KB
   tracks.parquet                    611.9 KB


---

## ✅ Done!

Your library is now saved locally. Next steps:

1. **Analyze**: Open `02_analyze_library.ipynb` for visualizations
2. **Playlist Analysis**: Open `03_playlist_analysis.ipynb` for genre clustering
3. **Re-sync**: Run this notebook again anytime to fetch new changes

The data is cached, so future runs are fast!